In [20]:
import os

import nibabel as nib
import numpy as np
import ants
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage import exposure

import matplotlib.pyplot as plt

we use 3D slicer to read `.nii` file and then export it as `.nii.gz` which has clear coordinate information

In [62]:
file = "/Users/yibeichen/Desktop/fusi_test_0725/fusi.nii.gz"
img = nib.load(file)
data = img.get_fdata()

print(data.shape)

affine = img.affine

# Create a transformation matrix to swap x and y axes in the affine matrix
# This involves swapping the first and second columns of the affine matrix
transform_matrix = np.array([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, -1, 0],
                            [0, 0, 0, 1]])

transformed_affine = affine @ transform_matrix
print(transformed_affine) 

flipped_data = np.flip(data, axis=2)

new_img = nib.Nifti1Image(flipped_data, transformed_affine, img.header)

output_file = "/Users/yibeichen/Desktop/fusi_test_0725/fusi_corrected.nii.gz"
# nib.save(new_img, output_file)

(192, 151, 173)
[[0.11    0.      0.      0.     ]
 [0.      0.2     0.      0.     ]
 [0.      0.      0.09856 0.     ]
 [0.      0.      0.      1.     ]]


In [81]:
file = "/Users/yibeichen/Desktop/fusi_test_0725/Charmander_3D.nii"
img = nib.load(file)
data = img.get_fdata()

data.shape,img.affine

pixdim[1,2,3] should be positive; setting to abs of pixdim values


((192, 46, 183),
 array([[-0.11      ,  0.        ,  0.        , 10.50499994],
        [ 0.        ,  0.2       ,  0.        , -4.50000007],
        [ 0.        ,  0.        ,  0.09856   , -8.96895982],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

In [85]:
file = "/Users/yibeichen/Desktop/fusi_test_0725/Charmander_3D.nii.gz"
img = nib.load(file)
data = img.get_fdata()

data.shape,img.affine, aff2axcodes(img.affine)

((192, 46, 183),
 array([[ 0.11   ,  0.     ,  0.     , -0.     ],
        [ 0.     ,  0.2    ,  0.     , -0.     ],
        [ 0.     ,  0.     , -0.09856,  0.     ],
        [ 0.     ,  0.     ,  0.     ,  1.     ]]),
 ('R', 'A', 'I'))

In [84]:
# correct orientation S/I


('R', 'A', 'I')

In [92]:
file = "/Users/yibeichen/Desktop/fusi/raw_data/Zion_3D.nii.gz"
img = nib.load(file)
data = img.get_fdata()

affine = img.affine

corrected_affine = affine.copy()
corrected_affine[[0, 1]] = corrected_affine[[1, 0]]

print("Corrected Affine:\n", corrected_affine)
print(aff2axcodes(corrected_affine))
# reflect z to be positive
transform_matrix = np.array([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, -1, 0],
                            [0, 0, 0, 1]])

transformed_affine = corrected_affine @ transform_matrix

print(transformed_affine)

# Create a new affine matrix for RAS orientation
ras_affine = np.zeros_like(transformed_affine)

# Copy the necessary elements to match RAS
ras_affine[0, :] = transformed_affine[1, :]  # Right
ras_affine[1, :] = transformed_affine[0, :]  # Anterior
ras_affine[2, :] = transformed_affine[2, :]  # Superior

# The last row should remain the same (for homogeneous coordinates)
ras_affine[3, :] = transformed_affine[3, :]

print("RAS Affine:\n", ras_affine)

# flip A/P (data was wrong) and S/I(match transformed affine)
flipped_data = np.flip(np.flip(np.flip(data, axis=0), axis=1), axis=2)
# flipped_data = np.flip(np.flip(data, axis=0), axis=2)

new_img = nib.Nifti1Image(flipped_data, transformed_affine, img.header)
print(new_img.get_fdata().shape)

output_file = "/Users/yibeichen/Desktop/fusi/raw_data/Zion_3D_corrected.nii.gz"
# nib.save(new_img, output_file)

Corrected Affine:
 [[ 0.       0.2      0.      -0.     ]
 [ 0.11     0.       0.      -0.     ]
 [ 0.       0.      -0.09856  0.     ]
 [ 0.       0.       0.       1.     ]]
('A', 'R', 'I')
[[0.      0.2     0.      0.     ]
 [0.11    0.      0.      0.     ]
 [0.      0.      0.09856 0.     ]
 [0.      0.      0.      1.     ]]
RAS Affine:
 [[0.11    0.      0.      0.     ]
 [0.      0.2     0.      0.     ]
 [0.      0.      0.09856 0.     ]
 [0.      0.      0.      1.     ]]
(128, 37, 153)


In [90]:
from nibabel.orientations import aff2axcodes

# Load the NIfTI file
nifti_path = "/Users/yibeichen/Desktop/fusi_test_0725/fus2mri/fusi_corrected-transformed.nii.gz"
img = nib.load(nifti_path)
data = img.get_fdata()
affine = img.affine
orientation = aff2axcodes(affine)
print(orientation)

# Determine the coordinates for the anterior right quarter
x_mid = data.shape[0] // 2
# y_mid = data.shape[1] // 2

# Extract the anterior right quarter
anterior_right_quarter = data[x_mid:, :, :]
print(anterior_right_quarter.shape)

new_affine = affine.copy()

# Adjust the origin based on the new subvolume
new_affine[0, 3] += x_mid * affine[0, 0]
# new_affine[1, 3] += y_mid * affine[1, 1]

print(new_affine)
# # Create a new NIfTI image with the extracted data and adjusted affine
new_img = nib.Nifti1Image(anterior_right_quarter, new_affine)

# # Save the new NIfTI file
new_nifti_path = "/Users/yibeichen/Desktop/fusi_test_0725/fusi_corrected-transformed_R.nii.gz"
nib.save(new_img, new_nifti_path)

# print("Anterior right quarter of the NIfTI file saved successfully.")


('R', 'A', 'S')
(74, 200, 135)
[[ 2.00000003e-01  0.00000000e+00 -0.00000000e+00 -1.63912773e-07]
 [ 0.00000000e+00  2.00000003e-01 -0.00000000e+00 -1.44000006e+01]
 [ 0.00000000e+00  0.00000000e+00  2.00000003e-01 -3.60000014e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
